# File para meter os dados da colecao onde a stora foi buscar os de treino quebra de linha ao escrever no "ou_data.txt"

### Em principio não há duplicados com os do "train.txt" que a stora deu

In [54]:
import pandas as pd
import re

## Sacar os dados do csv

In [55]:
# Vetor com os gêneros permitidos
allowed_genres = ["drama", "comedy", "horror", "action", "romance", "western", "animation", "crime", "sci-fi"]

# Ler o arquivo CSV
df = pd.read_csv('/Users/pedromiguel/Downloads/wiki_movie_plots_deduped.csv')

print("Colunas:", df.columns)

Colunas: Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')


## Filtrar os dados em que os generos sao aqueles que o modelo sabe que existem

In [56]:
# Selecionar as colunas "Title", "Genre" e "Plot"
selected_columns = df[['Title', 'Origin/Ethnicity','Genre','Director', 'Plot']]

# Filtrar os valores da coluna "Genre" que estão no vetor 'allowed_genres'
filtered_data = selected_columns[selected_columns['Genre'].isin(allowed_genres)]

# Converter as colunas filtradas em uma lista de listas (vetor)
data_list = filtered_data.values.tolist()

## Ler os plots que a stora deu

In [57]:
dados_filmes = []
with open("./data/train.txt", 'r', encoding='utf-8') as file:
    # Lendo o arquivo linha por linha
    for linha in file:
        # Remover espaços em branco no início e fim (incluindo quebras de linha)
        linha = linha.strip()
        # Separar os dados usando o tab ('\t') como delimitador
        filme = linha.split('\t')
        # Adicionar a lista de filmes ao array 2D
        dados_filmes.append(filme)
        
# print(len(dados_filmes))

## Criar dados que nao existem no ficheiro que a stora  deu para treino

In [58]:
# Criar um novo array para armazenar dados não duplicados
novo_array = []

# Adicionar os dados do CSV ao novo_array se não existirem em dados_filmes
for row in data_list:
    title, origem, genre, diretor,plot = row[0], row[1], row[2], row[3], row[4]
    
    # Verificar se o título, gênero e plot já estão no array 'dados_filmes'
    if not any(filme[0] == title and filme[2] == genre and filme[1] == origem and filme[3]== diretor for filme in dados_filmes):
        novo_array.append([title, genre, plot])  # Adicionar ao novo array se não for duplicado

## Alterar a info dos plot, pq há uns que teem \n, e depois ele assume que é mesmo uma quebra de linha ao escrever no ficheiro de teste

In [59]:
# print(novo_array[7358])

clean_array = []
for filme in novo_array:
    # Remover os caracteres "\r", "\n", "\t" de cada string
    plot = filme[2].replace("\r", "").replace("\n", "").replace("\t", "")
    
    # Remover qualquer formato que contenha um dígito dentro de []
    plot = re.sub(r'\[\d+\]', '', plot)
    
    # Substituir "\'s" por "´s"
    plot = plot.replace("'s", "´s")
    
    # Substituir ".[\d]" por "."
    plot = re.sub(r'\.\[\d+\]', '.', plot)
    
    # Substituir todas as ocorrências de \' por '
    plot = re.sub(r"\\'", "'", plot)
    
    # Adicionar o filme limpo ao novo array
    clean_array.append([filme[0], filme[1], plot])

# print(clean_array[7358])

## Escrever no ficheiro "our_data.txt" os novos dados de teste

In [60]:
# Caminho do arquivo de saída
output_file = './data/our_data.txt'

# Abrir o arquivo para escrita
with open(output_file, 'w', encoding='utf-8') as file:
    for row in clean_array:
        # Escrever cada linha no arquivo, separando as colunas por '\t'
        file.write('\t'.join(row) + '\n')